In [1]:
from pandas.core.interchange.dataframe_protocol import DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, Series
import numpy as np
from sklearn.cluster import DBSCAN
from umap import UMAP
from sklearn.manifold import trustworthiness
import optuna


C:\Users\achim\Desktop\OSMI_Mental_Health\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# From https://www.commonwealthfund.org/publications/scorecard/2025/jun/2025-scorecard-state-health-system-performance
state_rankings = {
    "Massachusetts": 1, "Connecticut": 12, "New Jersey": 16, "Virginia": 14, "Vermont": 8,
    "New Hampshire": 3, "Minnesota": 15, "Washington": 9, "Rhode Island": 5, "Utah": 17,
    "New York": 7, "Wisconsin": 27, "California": 20, "Colorado": 10, "Illinois": 26,
    "Nebraska": 24, "Oregon": 13, "Pennsylvania": 13, "Hawaii": 2, "Iowa": 18, "Maryland": 8,
    "Maine": 21, "Delaware": 25, "North Dakota": 22, "Michigan": 23, "District of Columbia": 6,
    "Idaho": 19, "Montana": 28, "North Carolina": 29, "Ohio": 30, "New Mexico": 31,
    "South Dakota": 32, "Kansas": 33, "Arizona": 34, "Indiana": 35, "South Carolina": 36,
    "Kentucky": 37, "Alaska": 38, "Florida": 39, "Wyoming": 40, "Louisiana": 41,
    "Alabama": 42, "Missouri": 43, "Tennessee": 44, "Georgia": 45, "Nevada": 46,
    "West Virginia": 47, "Arkansas": 48, "Oklahoma": 49, "Texas": 50, "Mississippi": 51
}

state_abbreviations = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
}

In [25]:
col_1 = "Do you currently have a mental health disorder?"
col_2 = "What US state or territory do you live in?"

df = pd.read_pickle(r"data/edited/df_25_Prozent_fehlende_Werte_entfernt.pkl").loc[:, [col_1, col_2]]

In [14]:
df[col_1].value_counts(dropna=False)

Do you currently have a mental health disorder?
Yes      429
No       355
Maybe    227
Name: count, dtype: Int64

In [26]:
mapping_func = {
    "Yes": 2,
    "Maybe": 1,
    "No": 0
}

df[col_1] = df[col_1].map(mapping_func)

In [27]:
result = df.groupby(col_2).agg(
    Durchschnitts_Score=pd.NamedAgg(column=col_1, aggfunc="mean")
).reset_index()

In [28]:
result

,What US state or territory do you live in?,Durchschnitts_Score
0,Alabama,1.333333
1,Alaska,0.500000
2,Arizona,0.500000
3,California,1.181818
4,Colorado,1.380952
5,Connecticut,1.200000
6,Delaware,2.000000
7,District of Columbia,2.000000
8,Florida,1.000000
9,Georgia,1.000000


In [33]:
import plotly.express as px


result['abbr'] = result.iloc[:, 0].map(state_abbreviations)

# Plotly Karte
fig = px.choropleth(
    result,
    locations='abbr',            # <- Kürzel statt vollem Namen
    locationmode="USA-states",
    color='Durchschnitts_Score',
    scope="usa",
    color_continuous_scale="RdBu_r",
    title="Ranking der US-Bundesstaaten nach Umfrage-Wert"
)
fig.show()

In [30]:
import plotly.express as px

df_ranking = pd.DataFrame(state_rankings.items(), columns=['state', 'ranking'])
df_ranking['abbr'] = df_ranking['state'].map(state_abbreviations)

# Plotly Karte
fig = px.choropleth(
    df_ranking,
    locations='abbr',            # <- Kürzel statt vollem Namen
    locationmode="USA-states",
    color='ranking',
    scope="usa",
    color_continuous_scale="RdBu_r",
    title="Ranking der US-Bundesstaaten nach offiziellem Health Index"
)
fig.show()

In [36]:
result["Durchschnitts_Score"].corr(df_ranking["ranking"], method="pearson")

np.float64(0.01067348150857889)